In [71]:
import csv
import math
import os
import cv2
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw, ImageEnhance
import numpy as np
import pandas as pd
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as preprocess_input_ir
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as preprocess_input_mv
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.layers import Conv2D, Reshape, DepthwiseConv2D, ZeroPadding2D
from keras.utils import Sequence
from tensorflow.keras.backend import epsilon
from keras import Sequential
from keras.layers import Dense

from sklearn.model_selection import train_test_split

DATA_DIR = "images"

# 0.35, 0.5, 0.75, 1.0, 1.3, 1.4
ALPHA = 1.3

# 96, 128, 160, 192, 224
IMAGE_SIZE = 299

EPOCHS = 200
BATCH_SIZE = 24
PATIENCE = 50

MULTI_PROCESSING = True
THREADS = 12

data = pd.read_csv('training.csv')
train, test = train_test_split(data, test_size=0.1, random_state=123)

train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

train.to_csv('train.csv', index=False)
test.to_csv('validation.csv', index=False)

TRAIN_CSV = "train.csv"
VALIDATION_CSV = "validation.csv"

In [6]:

class DataGenerator(Sequence):

    def __init__(self, csv_file,rnd_rescale=True, rnd_multiply=True, rnd_color=True, rnd_crop=True, rnd_flip=False,
                 batch_size = BATCH_SIZE):
        self.paths = []
        self.coords = []
        self.batch_size = batch_size
        self.rnd_rescale = rnd_rescale
        self.rnd_multiply = rnd_multiply
        self.rnd_color = rnd_color
        self.rnd_crop = rnd_crop
        self.rnd_flip = rnd_flip

        with open(csv_file, "r") as file:
            self.coords = np.zeros((sum(1 for line in file)-1, 4))
            
        df = pd.read_csv(csv_file)

        for index, row in df.iterrows():
            path = os.path.join(DATA_DIR, row['image_name'])
            x1 = row['x1']
            x2 = row['x2']
            y1 = row['y1']
            y2 = row['y2']

            img = Image.open(path)
            width, height = img.size

            self.coords[index, 0] = x1
            self.coords[index, 1] = y1
            self.coords[index, 2] = x2 
            self.coords[index, 3] = y2 

            self.paths.append(path)

    def __len__(self):
        return math.ceil(len(self.coords) / self.batch_size)

    def __getitem__(self, idx):
        batch_paths = self.paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_coords = self.coords[idx * self.batch_size:(idx + 1) * self.batch_size].copy()

        batch_images = np.zeros((len(batch_paths), IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
        for i, f in enumerate(batch_paths):
            img = Image.open(f)
            x0,y0,x1,y1 = batch_coords[i]
            if self.rnd_rescale:
                old_width = img.width
                old_height = img.height

                rescale = np.random.uniform(low=0.6, high=1.4)
                new_width = int(old_width * rescale)
                new_height = int(old_height * rescale)

                img = img.resize((new_width, new_height))

                x0 *= new_width / old_width
                y0 *= new_height / old_height
                x1 *= new_width / old_width
                y1 *= new_height / old_height

            if self.rnd_crop:
                start_x = np.random.randint(0, high=np.floor(0.15 * img.width))
                stop_x = img.width - np.random.randint(start_x, high=np.floor(0.15 * img.width))
                start_y = np.random.randint(0, high=np.floor(0.15 * img.height))
                stop_y = img.height - np.random.randint(start_y, high=np.floor(0.15 * img.height))

                img = img.crop((start_x, start_y, stop_x, stop_y))

                x0 = max(x0 - start_x, 0)
                y0 = max(y0 - start_y, 0)
                x1 = min(x1 - start_x, img.width)
                y1 = min(y1 - start_y, img.height)

                if np.abs(x1 - x0) < 5 or np.abs(y1 - y0) < 5:
                    print("\nWarning: cropped too much (obj width {}, obj height {}, img width {}, img height {})\n".format(x1 - x0, y1 - y0, img.width, img.height))

            if self.rnd_flip:
                elem = np.random.choice([0, 90, 180, 270, 1423, 1234])
                if elem % 10 == 0:
                    x = x0 - img.width / 2
                    y = y0 - img.height / 2

                    x0 = img.width / 2 + x * np.cos(np.deg2rad(elem)) - y * np.sin(np.deg2rad(elem))
                    y0 = img.height / 2 + x * np.sin(np.deg2rad(elem)) + y * np.cos(np.deg2rad(elem))

                    x = x1 - img.width / 2
                    y = y1 - img.height / 2

                    x1 = img.width / 2 + x * np.cos(np.deg2rad(elem)) - y * np.sin(np.deg2rad(elem))
                    y1 = img.height / 2 + x * np.sin(np.deg2rad(elem)) + y * np.cos(np.deg2rad(elem))

                    img = img.rotate(-elem)
                else:
                    if elem == 1423:
                        img = img.transpose(Image.FLIP_TOP_BOTTOM)
                        y0 = img.height - y0
                        y1 = img.height - y1

                    elif elem == 1234:
                        img = img.transpose(Image.FLIP_LEFT_RIGHT)
                        x0 = img.width - x0
                        x1 = img.width - x1

            image_width = img.width
            image_height = img.height

            tmp = x0
            x0 = min(x0, x1)
            x1 = max(tmp, x1)

            tmp = y0
            y0 = min(y0, y1)
            y1 = max(tmp, y1)

            x0 = max(x0, 0)
            y0 = max(y0, 0)

            y0 = min(y0, image_height)
            x0 = min(x0, image_width)
            y1 = min(y1, image_height)
            x1 = min(x1, image_width)

            if self.rnd_color:
                enhancer = ImageEnhance.Color(img)
                img = enhancer.enhance(np.random.uniform(low=0.5, high=1.5))

                enhancer2 = ImageEnhance.Brightness(img)
                img = enhancer.enhance(np.random.uniform(low=0.7, high=1.3))

            img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
            img = img.convert('RGB')
            pil_img = img
            img = np.array(img, dtype=np.float32)
            pil_img.close()
            if self.rnd_multiply:
                img[...,0] = np.floor(np.clip(img[...,0] * np.random.uniform(low=0.8, high=1.2), 0.0, 255.0))
                img[...,1] = np.floor(np.clip(img[...,1] * np.random.uniform(low=0.8, high=1.2), 0.0, 255.0))
                img[...,2] = np.floor(np.clip(img[...,2] * np.random.uniform(low=0.8, high=1.2), 0.0, 255.0))

            batch_images[i] = preprocess_input(img.copy())
            
            batch_coords[i, 0] = x0 
            batch_coords[i, 1] = y0 
            batch_coords[i, 2] = (x1 - x0) 
            batch_coords[i, 3] = (y1 - y0)  

        return batch_images, batch_coords
            
        return np.concatenate((batch_images, batch_images_rotate90)), np.concatenate((batch_coords, batch_coords_rotate90))
class Validation(Callback):
    def __init__(self, generator):
        self.generator = generator

    def on_epoch_end(self, epoch, logs):
        mse = 0
        intersections = 0
        unions = 0

        for i in range(len(self.generator)):
            batch_images, gt = self.generator[i]
            pred = self.model.predict_on_batch(batch_images)
            mse += np.linalg.norm(gt - pred, ord='fro') / pred.shape[0]

            pred = np.maximum(pred, 0)

            diff_width = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
            diff_height = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
            intersection = np.maximum(diff_width, 0) * np.maximum(diff_height, 0)

            area_gt = gt[:,2] * gt[:,3]
            area_pred = pred[:,2] * pred[:,3]
            union = np.maximum(area_gt + area_pred - intersection, 0)

            intersections += np.sum(intersection * (union > 0))
            unions += np.sum(union)

        iou = np.round(intersections / (unions + epsilon()), 4)
        logs["val_iou"] = iou

        mse = np.round(mse, 4)
        logs["val_mse"] = mse

        print(" - val_iou: {} - val_mse: {}".format(iou, mse))

def create_irv2(trainable=False):
    model = InceptionResNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights=None)

    # to freeze layers
    for layer in model.layers:
        layer.trainable = False

    x = model.layers[-1].output
    x = Conv2D(512, kernel_size=4, name="coords1", activation='relu')(x)
    x = Conv2D(256, kernel_size=3, name="coords2", activation='relu')(x)
    x = Conv2D(4, kernel_size=3, name="coords3", activation='relu')(x)
    x = Reshape((4,))(x)

    return Model(inputs=model.input, outputs=x)

def create_mnv2(trainable=False):
    model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, alpha=ALPHA, weights=None)

    # to freeze layers
    for layer in model.layers:
        layer.trainable = False

    x = model.layers[-1].output
    x = Conv2D(256, kernel_size=3, name="coords1", activation='relu')(x)
    x = Conv2D(4, kernel_size=5, name="coords2", activation='relu')(x)
    x = Reshape((4,))(x)

    return Model(inputs=model.input, outputs=x)


In [ ]:
def ensemble_model(irv_model, mnv_model):
    irv_pred = irv_model.layers[-1].output
    irv_pred
    mnv_pred = mnv_model.layers[-1].output
    

In [29]:
mnv2 = create_mnv2()
mnv2.load_weights('mobilenetV2-no-weights-0.91.h5')


In [8]:
irv2 = create_irv2()
irv2.load_weights('inception_resnetV2-no_weights-freeze-0.91.h5')


In [9]:
irv2_pred = pd.read_csv("test.csv")
irv2_pred.head()

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,NaN,NaN,NaN,NaN
1,1473231475010DeeplearnS11276.png,NaN,NaN,NaN,NaN
2,JPEG_20161205_135307_1000155917326.png,NaN,NaN,NaN,NaN
3,JPEG_20160711_123440_1000518778437.png,NaN,NaN,NaN,NaN
4,JPEG_20160803_115329_100034020722.png,NaN,NaN,NaN,NaN


In [10]:
for index, row in irv2_pred.iterrows():
    
    unscaled = cv2.imread('images/'+row['image_name'])
    image_height, image_width, _ = unscaled.shape
    
    IMAGE_SIZE = 299
    
    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input_ir(np.array(image, dtype=np.float32))

    region = irv2.predict(x=np.array([feat_scaled]))[0]

    x1a = (region[0] * image_width / IMAGE_SIZE)
    y1a = (region[1] * image_height / IMAGE_SIZE)

    x2a = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2a = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    irv2_pred.iloc[index,1] = ((x1a)/2)
    irv2_pred.iloc[index,2] = ((x2a)/2)
    irv2_pred.iloc[index,3] = ((y1a)/2)
    irv2_pred.iloc[index,4] = ((y2a)/2)
    
    if (index%1000)==0:
        print(index)

irv2_pred.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


KeyboardInterrupt: 

In [30]:
mnv2_pred = pd.read_csv("test.csv")
mnv2_pred.head()

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,NaN,NaN,NaN,NaN
1,1473231475010DeeplearnS11276.png,NaN,NaN,NaN,NaN
2,JPEG_20161205_135307_1000155917326.png,NaN,NaN,NaN,NaN
3,JPEG_20160711_123440_1000518778437.png,NaN,NaN,NaN,NaN
4,JPEG_20160803_115329_100034020722.png,NaN,NaN,NaN,NaN


In [23]:
train_df = pd.read_csv("training.csv")
train_df.head()

,image_name,x1,x2,y1,y2
0,JPEG_20160706_121146_1000145715002.png,115,495,143,325
1,JPEG_20161119_174038_1000690577600.png,23,457,61,409
2,147444927651111470309333776-Roadster-Men-Casua...,37,601,13,470
3,147772332675720161028_161611.png,27,602,162,385
4,1473315333651DeeplearnS11638.png,60,586,174,325


In [33]:
df = pd.DataFrame(np.zeros((len(train_df), 13)))

In [35]:
for index, row in train_df.iterrows():
    
    unscaled = cv2.imread('images/'+row['image_name'])
    image_height, image_width, _ = unscaled.shape
    
    IMAGE_SIZE = 299
    
    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input_ir(np.array(image, dtype=np.float32))

    region = irv2.predict(x=np.array([feat_scaled]))[0]

    x1a = (region[0] * image_width / IMAGE_SIZE)
    y1a = (region[1] * image_height / IMAGE_SIZE)

    x2a = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2a = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    IMAGE_SIZE = 224

    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input_mv(np.array(image, dtype=np.float32))

    region = mnv2.predict(x=np.array([feat_scaled]))[0]

    x1b = (region[0] * image_width / IMAGE_SIZE)
    y1b = (region[1] * image_height / IMAGE_SIZE)

    x2b = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2b = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    df.iloc[index,0] = row['image_name']
    df.iloc[index,1] = ((x1a))
    df.iloc[index,2] = ((x2a))
    df.iloc[index,3] = ((y1a))
    df.iloc[index,4] = ((y2a))
    df.iloc[index,5] = ((x1b))
    df.iloc[index,6] = ((x2b))
    df.iloc[index,7] = ((y1b))
    df.iloc[index,8] = ((y2b))
    df.iloc[index,9] = row['x1']
    df.iloc[index,10] = row['x2']
    df.iloc[index,11] = row['y1']
    df.iloc[index,12] = row['y2']
    
    if (index%1000)==0:
        print(index)

df.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,JPEG_20160706_121146_1000145715002.png,114.592040,485.142500,145.374205,325.247488,111.899054,490.973990,143.549995,318.539690,115.0,495.0,143.0,325.0
1,JPEG_20161119_174038_1000690577600.png,25.396328,459.897020,62.030487,411.908624,14.900318,458.294285,57.182395,411.727916,23.0,457.0,61.0,409.0
2,147444927651111470309333776-Roadster-Men-Casua...,30.202265,593.225792,12.857371,463.697808,38.461993,584.657026,12.636195,458.999241,37.0,601.0,13.0,470.0
3,147772332675720161028_161611.png,24.748062,602.583612,161.349796,383.408718,27.214124,598.370536,164.150211,382.583902,27.0,602.0,162.0,385.0
4,1473315333651DeeplearnS11638.png,60.361998,586.745336,175.362128,327.451221,59.576765,587.784816,173.057213,328.764561,60.0,586.0,174.0,325.0


In [36]:
df.to_csv('training2.csv')

In [38]:
val_df = pd.read_csv("validation.csv")
val_df.head()

,image_name,x1,x2,y1,y2
0,JPEG_20160624_135055_1000527160000.png,132,606,165,325
1,147771890839520161028_141911.png,123,544,52,452
2,1474715198275DSC07675.png,311,458,32,459
3,JPEG_20161117_112207_1000880661419.png,13,640,5,459
4,147444941744811473248953580-Roadster-Men-Navy-...,34,603,16,471


In [39]:
df2 =  pd.DataFrame(np.zeros((len(val_df), 13)))

In [40]:
for index, row in val_df.iterrows():
    
    unscaled = cv2.imread('images/'+row['image_name'])
    image_height, image_width, _ = unscaled.shape
    
    IMAGE_SIZE = 299
    
    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input_ir(np.array(image, dtype=np.float32))

    region = irv2.predict(x=np.array([feat_scaled]))[0]

    x1a = (region[0] * image_width / IMAGE_SIZE)
    y1a = (region[1] * image_height / IMAGE_SIZE)

    x2a = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2a = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    IMAGE_SIZE = 224

    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input_mv(np.array(image, dtype=np.float32))

    region = mnv2.predict(x=np.array([feat_scaled]))[0]

    x1b = (region[0] * image_width / IMAGE_SIZE)
    y1b = (region[1] * image_height / IMAGE_SIZE)

    x2b = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2b = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    df2.iloc[index,0] = row['image_name']
    df2.iloc[index,1] = ((x1a))
    df2.iloc[index,2] = ((x2a))
    df2.iloc[index,3] = ((y1a))
    df2.iloc[index,4] = ((y2a))
    df2.iloc[index,5] = ((x1b))
    df2.iloc[index,6] = ((x2b))
    df2.iloc[index,7] = ((y1b))
    df2.iloc[index,8] = ((y2b))
    df2.iloc[index,9] = row['x1']
    df2.iloc[index,10] = row['x2']
    df2.iloc[index,11] = row['y1']
    df2.iloc[index,12] = row['y2']
    
    if (index%1000)==0:
        print(index)

df2.head()

0
1000


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,JPEG_20160624_135055_1000527160000.png,132.316241,595.976432,173.387921,320.458870,140.286015,600.110038,173.996577,317.819759,132.0,606.0,165.0,325.0
1,147771890839520161028_141911.png,124.201832,549.382251,65.640745,459.768891,123.266852,543.815613,60.365218,453.204313,123.0,544.0,52.0,452.0
2,1474715198275DSC07675.png,305.480214,462.097976,25.053617,452.708393,304.909690,457.154105,31.033978,463.411200,311.0,458.0,32.0,459.0
3,JPEG_20161117_112207_1000880661419.png,130.057042,623.085676,53.449472,462.756976,0.000000,630.595921,0.000000,480.802100,13.0,640.0,5.0,459.0
4,147444941744811473248953580-Roadster-Men-Navy-...,36.165800,601.265808,28.245072,472.670098,36.505154,601.419939,20.901620,471.695088,34.0,603.0,16.0,471.0


In [41]:
df2.to_csv('validation2.csv')

In [31]:
for index, row in mnv2_pred.iterrows():
    
    unscaled = cv2.imread('images/'+row['image_name'])
    image_height, image_width, _ = unscaled.shape
    
    IMAGE_SIZE = 224
    
    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input(np.array(image, dtype=np.float32))

    region = mnv2.predict(x=np.array([feat_scaled]))[0]

    x1a = (region[0] * image_width / IMAGE_SIZE)
    y1a = (region[1] * image_height / IMAGE_SIZE)

    x2a = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2a = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    mnv2_pred.iloc[index,1] = ((x1a)/2)
    mnv2_pred.iloc[index,2] = ((x2a)/2)
    mnv2_pred.iloc[index,3] = ((y1a)/2)
    mnv2_pred.iloc[index,4] = ((y2a)/2)
    
    if (index%1000)==0:
        print(index)

mnv2_pred.head()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,116.467111,221.568952,43.920054,202.691803
1,1473231475010DeeplearnS11276.png,36.528827,283.681205,68.367147,175.827272
2,JPEG_20161205_135307_1000155917326.png,73.820283,247.922167,27.912044,218.412454
3,JPEG_20160711_123440_1000518778437.png,106.739142,233.520726,45.708279,206.211875
4,JPEG_20160803_115329_100034020722.png,68.932964,248.840724,23.868491,216.744935


In [44]:
for index, row in df.iterrows():
    print(row)
    break

0     JPEG_20160706_121146_1000145715002.png
1                                    114.592
2                                    485.142
3                                    145.374
4                                    325.247
5                                    111.899
6                                    490.974
7                                     143.55
8                                     318.54
9                                        115
10                                       495
11                                       143
12                                       325
Name: 0, dtype: object


In [80]:
class DataGenerator2(Sequence):

    def __init__(self, csv_file,batch_size = BATCH_SIZE):
        self.preds = []
        self.coords = []
        self.batch_size = batch_size
        
        with open(csv_file, "r") as file:
            nrows = sum(1 for line in file)
            self.coords = np.zeros((nrows, 4))
            self.preds = np.zeros((nrows,8))
            
        df = pd.read_csv(csv_file)

        for index, row in df.iterrows():
            x1 = row[10]
            x2 = row[11]
            y1 = row[12]
            y2 = row[13]


            self.coords[index, 0] = x1
            self.coords[index, 1] = y1
            self.coords[index, 2] = x2 
            self.coords[index, 3] = y2
            for i in range(2,10):
                self.preds[index,i-2] = row[i]


    def __len__(self):
        return math.ceil(len(self.coords) / self.batch_size)

    def __getitem__(self, idx):
        batch_coords = self.coords[idx * self.batch_size:(idx + 1) * self.batch_size].copy()

        batch_preds = self.preds[idx * self.batch_size:(idx + 1) * self.batch_size].copy()


        return batch_preds, batch_coords
            
class Validation2(Callback):
    def __init__(self, generator):
        self.generator = generator

    def on_epoch_end(self, epoch, logs):
        mse = 0
        intersections = 0
        unions = 0

        for i in range(len(self.generator)):
            batch_preds, gt = self.generator[i]
            pred = self.model.predict_on_batch(batch_preds)
            mse += np.linalg.norm(gt - pred, ord='fro') / pred.shape[0]

            pred = np.maximum(pred, 0)

            diff_width = np.minimum(gt[:,2], pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
            diff_height = np.minimum(gt[:,3], pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
            intersection = np.maximum(diff_width, 0) * np.maximum(diff_height, 0)

            area_gt = (gt[:,2] - gt[:,0]) * (gt[:,3] - gt[:,1])
            area_pred = (pred[:,2] - pred[:,0]) * (pred[:,3] - pred[:,1])
            union = np.maximum(area_gt + area_pred - intersection, 0)

            intersections += np.sum(intersection * (union > 0))
            unions += np.sum(union)

        iou = np.round(intersections / (unions + epsilon()), 4)
        logs["val_iou"] = iou

        mse = np.round(mse, 4)
        logs["val_mse"] = mse

        print(" - val_iou: {} - val_mse: {}".format(iou, mse))

In [87]:
model = Sequential()
model.add(Dense(4, input_shape = (8,)))

In [88]:

train_datagen = DataGenerator2('training2.csv')
validation_datagen = Validation2(generator=DataGenerator2('validation2.csv',batch_size = 256))

model.compile(loss="mean_absolute_error", optimizer="adam", metrics=['accuracy'])

checkpoint = ModelCheckpoint("model-{val_iou:.2f}.h5", monitor="val_iou", verbose=1, save_best_only=True,
                             save_weights_only=True, mode="max", period=1)
#stop = EarlyStopping(monitor="val_iou", patience=PATIENCE, mode="max")
#reduce_lr = ReduceLROnPlateau(monitor="val_iou", factor=0.2, patience=10, min_lr=1e-7, verbose=1, mode="max")

# model.summary()



In [89]:
model.fit_generator(generator=train_datagen,
                    epochs=EPOCHS,
                    callbacks=[validation_datagen],#, reduce_lr, stop, checkpoint],
                    workers=THREADS,
                    use_multiprocessing=MULTI_PROCESSING,
                    shuffle=True,
                    verbose=1)


Epoch 1/200
584/584 [==============================] - 16s 28ms/step - loss: 110.4703 - acc: 0.5786
Epoch 1/200
 - val_iou: 0.7298 - val_mse: 40.4106
Epoch 2/200
584/584 [==============================] - 2s 4ms/step - loss: 23.4575 - acc: 0.9031
 - val_iou: 0.8132 - val_mse: 29.013
Epoch 3/200
584/584 [==============================] - 4s 6ms/step - loss: 15.9269 - acc: 0.9260
 - val_iou: 0.8456 - val_mse: 24.677200000000003
Epoch 4/200
584/584 [==============================] - 3s 6ms/step - loss: 12.7015 - acc: 0.9435
 - val_iou: 0.8656 - val_mse: 20.727
Epoch 5/200
584/584 [==============================] - 4s 6ms/step - loss: 9.9684 - acc: 0.9590
 - val_iou: 0.8864000000000001 - val_mse: 17.497700000000002
Epoch 6/200
584/584 [==============================] - 3s 6ms/step - loss: 7.3491 - acc: 0.9731
 - val_iou: 0.9026000000000001 - val_mse: 16.1685
Epoch 7/200
584/584 [==============================] - 3s 6ms/step - loss: 6.1061 - acc: 0.9795
 - val_iou: 0.9032 - val_mse: 16.1373

584/584 [==============================] - 4s 6ms/step - loss: 4.4427 - acc: 0.9838
 - val_iou: 0.9143 - val_mse: 15.089
Epoch 59/200
584/584 [==============================] - 4s 6ms/step - loss: 4.4204 - acc: 0.9844
 - val_iou: 0.9141 - val_mse: 15.077300000000001
Epoch 60/200
584/584 [==============================] - 3s 6ms/step - loss: 4.4127 - acc: 0.9841
 - val_iou: 0.914 - val_mse: 15.086
Epoch 61/200
584/584 [==============================] - 3s 6ms/step - loss: 4.4067 - acc: 0.9840
 - val_iou: 0.9143 - val_mse: 15.037600000000001
Epoch 62/200
584/584 [==============================] - 4s 6ms/step - loss: 4.3997 - acc: 0.9846
 - val_iou: 0.9145000000000001 - val_mse: 15.0654
Epoch 63/200
584/584 [==============================] - 3s 6ms/step - loss: 4.3826 - acc: 0.9845
 - val_iou: 0.914 - val_mse: 15.0492
Epoch 64/200
584/584 [==============================] - 4s 6ms/step - loss: 4.3827 - acc: 0.9845
 - val_iou: 0.914 - val_mse: 15.087800000000001
Epoch 65/200
584/584 [======

584/584 [==============================] - 3s 6ms/step - loss: 4.1575 - acc: 0.9850
 - val_iou: 0.9143 - val_mse: 14.966700000000001
Epoch 116/200
584/584 [==============================] - 3s 6ms/step - loss: 4.1666 - acc: 0.9855
 - val_iou: 0.9154 - val_mse: 14.9679
Epoch 117/200
584/584 [==============================] - 3s 6ms/step - loss: 4.1540 - acc: 0.9855
 - val_iou: 0.9151 - val_mse: 14.9859
Epoch 118/200
584/584 [==============================] - 4s 6ms/step - loss: 4.1471 - acc: 0.9851
 - val_iou: 0.9155000000000001 - val_mse: 14.966000000000001
Epoch 119/200
584/584 [==============================] - 4s 6ms/step - loss: 4.1431 - acc: 0.9858
 - val_iou: 0.9155000000000001 - val_mse: 14.9791
Epoch 120/200
584/584 [==============================] - 3s 6ms/step - loss: 4.1464 - acc: 0.9852
 - val_iou: 0.9153 - val_mse: 14.999500000000001
Epoch 121/200
584/584 [==============================] - 4s 6ms/step - loss: 4.1538 - acc: 0.9858
 - val_iou: 0.9152 - val_mse: 15.0089
Epoch

584/584 [==============================] - 4s 6ms/step - loss: 4.1171 - acc: 0.9852
 - val_iou: 0.9149 - val_mse: 15.002600000000001
Epoch 172/200
584/584 [==============================] - 3s 6ms/step - loss: 4.1303 - acc: 0.9859
 - val_iou: 0.9146000000000001 - val_mse: 15.012300000000002
Epoch 173/200
584/584 [==============================] - 4s 6ms/step - loss: 4.1191 - acc: 0.9857
 - val_iou: 0.9148000000000001 - val_mse: 15.0286
Epoch 174/200
584/584 [==============================] - 3s 6ms/step - loss: 4.1058 - acc: 0.9856
 - val_iou: 0.9148000000000001 - val_mse: 14.989600000000001
Epoch 175/200
584/584 [==============================] - 4s 6ms/step - loss: 4.1190 - acc: 0.9860
 - val_iou: 0.9146000000000001 - val_mse: 15.0029
Epoch 176/200
584/584 [==============================] - 4s 6ms/step - loss: 4.1255 - acc: 0.9856
 - val_iou: 0.9152 - val_mse: 14.992600000000001
Epoch 177/200
584/584 [==============================] - 4s 6ms/step - loss: 4.1249 - acc: 0.9858
 - val_i

In [90]:
model.get_weights()

[array([[ 0.8810862 ,  0.01069998, -0.05550295, -0.02674494],
        [-0.03702207, -0.00653689,  0.7570125 ,  0.0156884 ],
        [ 0.0246113 ,  0.9205344 , -0.03372381, -0.09611034],
        [-0.01624546, -0.02390493,  0.01695321,  0.7706424 ],
        [ 0.11937341, -0.00885947,  0.05389476,  0.03211689],
        [ 0.03704379,  0.00728715,  0.24110231, -0.01620385],
        [-0.02487618,  0.07641736,  0.02889517,  0.09707145],
        [ 0.01761523,  0.02381223, -0.02079938,  0.2209271 ]],
       dtype=float32),
 array([ 0.29131022, -0.08184507,  1.5695745 ,  1.9415376 ], dtype=float32)]

In [ ]:
for index, row in sample.iterrows():
    
    unscaled = cv2.imread('images/'+row['image_name'])
    image_height, image_width, _ = unscaled.shape
    
    IMAGE_SIZE = 299
    
    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input(np.array(image, dtype=np.float32))

    region = irv2.predict(x=np.array([feat_scaled]))[0]

    x1a = (region[0] * image_width / IMAGE_SIZE)
    y1a = (region[1] * image_height / IMAGE_SIZE)

    x2a = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2a = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    IMAGE_SIZE = 224

    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input(np.array(image, dtype=np.float32))

    region = mnv2.predict(x=np.array([feat_scaled]))[0]

    x1b = (region[0] * image_width / IMAGE_SIZE)
    y1b = (region[1] * image_height / IMAGE_SIZE)

    x2b = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2b = ((region[1] + region[3]) * image_height / IMAGE_SIZE)


    sample.iloc[index,1] = int((x1a + x1b)/2)
    sample.iloc[index,2] = int((x2a + x2b)/2)
    sample.iloc[index,3] = int((y1a + y1b)/2)
    sample.iloc[index,4] = int((y2a + y2b)/2)

sample.head()

In [ ]:
sample.to_csv('prediction.csv', index=False)

In [34]:
irv2_pred.head()

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,116.658629,221.043926,44.399907,204.169738
1,1473231475010DeeplearnS11276.png,37.768351,283.908700,71.241298,175.949830
2,JPEG_20161205_135307_1000155917326.png,73.168774,248.297091,27.214445,216.793348
3,JPEG_20160711_123440_1000518778437.png,106.113224,233.519218,46.081299,208.614302
4,JPEG_20160803_115329_100034020722.png,68.700429,248.980717,21.831913,213.386459


In [35]:
mnv2_pred.head()

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,116.467111,221.568952,43.920054,202.691803
1,1473231475010DeeplearnS11276.png,36.528827,283.681205,68.367147,175.827272
2,JPEG_20161205_135307_1000155917326.png,73.820283,247.922167,27.912044,218.412454
3,JPEG_20160711_123440_1000518778437.png,106.739142,233.520726,45.708279,206.211875
4,JPEG_20160803_115329_100034020722.png,68.932964,248.840724,23.868491,216.744935


In [ ]:
df_concat = pd.concat((mnv2_pred, irv2_pred))

In [41]:
by_row_index = df_concat.groupby(df_concat.index)
df_means = by_row_index.mean(numeric_only=True)

In [42]:
df_means.head()

,x1,x2,y1,y2
0,116.562870,221.306439,44.159980,203.430771
1,37.148589,283.794952,69.804223,175.888551
2,73.494528,248.109629,27.563245,217.602901
3,106.426183,233.519972,45.894789,207.413088
4,68.816696,248.910721,22.850202,215.065697


In [46]:
df_means['image_name'] = irv2_pred['image_name']

In [47]:
df_means.head()

,x1,x2,y1,y2,image_name
0,116.562870,221.306439,44.159980,203.430771,1474723840903DSC08089.png
1,37.148589,283.794952,69.804223,175.888551,1473231475010DeeplearnS11276.png
2,73.494528,248.109629,27.563245,217.602901,JPEG_20161205_135307_1000155917326.png
3,106.426183,233.519972,45.894789,207.413088,JPEG_20160711_123440_1000518778437.png
4,68.816696,248.910721,22.850202,215.065697,JPEG_20160803_115329_100034020722.png


In [48]:
cols = df_means.columns.tolist()

In [49]:
cols = cols[-1:] + cols[:-1]
df_means = df_means[cols]

In [50]:
df_means.head()

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,116.562870,221.306439,44.159980,203.430771
1,1473231475010DeeplearnS11276.png,37.148589,283.794952,69.804223,175.888551
2,JPEG_20161205_135307_1000155917326.png,73.494528,248.109629,27.563245,217.602901
3,JPEG_20160711_123440_1000518778437.png,106.426183,233.519972,45.894789,207.413088
4,JPEG_20160803_115329_100034020722.png,68.816696,248.910721,22.850202,215.065697


In [51]:
df_means.to_csv('prediction_decimal.csv', index=False)

In [54]:
df_means[['x1','x2','y1','y2']] = df_means[['x1','x2','y1','y2']].astype('int64')

In [55]:
df_means.head()

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,116,221,44,203
1,1473231475010DeeplearnS11276.png,37,283,69,175
2,JPEG_20161205_135307_1000155917326.png,73,248,27,217
3,JPEG_20160711_123440_1000518778437.png,106,233,45,207
4,JPEG_20160803_115329_100034020722.png,68,248,22,215


In [56]:
df_means.to_csv('prediction_int.csv', index=False)